In [1]:
from dataclasses import asdict
import json

from IPython.display import display, Markdown
import pandas as pd

import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
from deepsearch.cps.client.queries import Query

from deepsearch.core.client import DeepSearchKeyAuth, DeepSearchConfig
from deepsearch.cps.client.api import CpsApi, CpsApiClient

## CPS Authentication

In [3]:
auth_filename = "cps-auth.json" # this file should be populated with the credentials
auth_data = json.load(open(auth_filename))

auth = DeepSearchKeyAuth(
    username=auth_data["email"],
    api_key=auth_data["api_key"],
)

config = DeepSearchConfig(
    host="https://cps.foc-deepsearch.zurich.ibm.com", # IBM internal system
    auth=auth,
)

client = CpsApiClient(config)
api = CpsApi(client)

## Select KG

In [4]:
# Select KG
kg = api.knowledge_graphs.get(
    "f21574fe745fa0b8213cb08d0c4166513108158c", # proj_key
    "80511ed4a524fe446dc102e704fd7577f2c2a26c", # bag_key
)

## Load the WF query

The file `./workflows/keywords_from_category.json` was created from the code snippet generated in the CPS UI.

In [5]:
query_filename = './workflows/keywords_from_category.json'
wf_query = json.load(open(query_filename))

query = Query()
wf = query.add(
    "Workflow",
    parameters={
        "workflow": wf_query['template']
    },
    coordinates=kg
)

for k, spec in wf_query.get('outputs', {}).items():
    wf.output(k).output_as(spec['name'])

## Execute and print results

In [6]:
%%time
# Execute
result = api.queries.run(query)

CPU times: user 24.1 ms, sys: 6.14 ms, total: 30.2 ms
Wall time: 545 ms


In [7]:
# Timings
print('Query execution took', result.timings.overall, 'sec')

Query execution took 0.14111201278865337 sec


In [8]:
# Visualize the outputs as table
for name, output in result.outputs.items():
    df = pd.json_normalize(output)
    display(Markdown(f"## Output '{name}'"))
    display(df)

## Output 'keyphrases'

,_categories,_db,_hash,_id,_name,_synonyms,index,weight
0,[key-phrases],6650739843425458168,e13cfc4b2b4f22fefaa2e7f0302d2af8,617fca9aa4557eb20cdb0c6f,challenge,[challenge],215396,154
1,[key-phrases],6650739843425458168,9f8972945e41640618a0691137f367be,617fca96fcf995f4a3497520,text,[text],211746,134
2,[key-phrases],6650739843425458168,e460f5c2530cb8541a08694e94e38200,617fca9718c78284f1a4f699,data,[data],212193,108
3,[key-phrases],6650739843425458168,2d85965fa308000804b75ebf759d6bff,617fca98dff425e9629a1a08,models,[models],212879,102
4,[key-phrases],6650739843425458168,49aea03140d0938a54b25104b6ae38ed,617fca99fa0ff72c11494587,work,[work],214361,98
...,...,...,...,...,...,...,...,...
1019,[key-phrases],6650739843425458168,d051cad28025f24444d537c4cb0d917e,617fca9a9d5607e5295b0b9c,manner,[manner],215386,3
1020,[key-phrases],6650739843425458168,3903ab6f758f2372d313ad2d03a0bef0,617fca9a1164905d91924c48,number,[number],215435,3
1021,[key-phrases],6650739843425458168,e597d05511968cecc99f31713d0d8a7c,617fca9aa4557eb20cdb0c75,explanations,[explanations],215582,3
1022,[key-phrases],6650739843425458168,89bd252e86ee4bb3106f0fec74a7bc32,617fca9a8b0915e5eca531c9,textual entailment,[textual entailment],215605,3
